<a href="https://colab.research.google.com/github/mayankpara/deep-learning/blob/main/Text-Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import keras
import numpy as np

In [ ]:
# load data
path = keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

606208/600901 [==============================] - 0s 1us/step


In [ ]:
# open data
text = open(path).read().lower()
print('corpus length:',len(text))

corpus length: 600893


# Vectorizing **data** 

In [ ]:
# vectorizing sequences of data
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0,len(text)-maxlen,step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print('number of sequences:',len(sentences))
chars = sorted(list(set(text)))
char_indices = dict((char,chars.index(char)) for char in chars)
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
  for t,char in enumerate(sentence):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_chars[i]]] = 1

number of sequences: 200278


In [ ]:
# single layer LSTM model
model = keras.models.Sequential([
                                 keras.layers.LSTM(128,input_shape=(maxlen,len(chars))),
                                 keras.layers.Dense(len(chars),activation='softmax')
])

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=optimizer,loss='categorical_crossentropy')

In [ ]:
# function to sample next character
def sample(preds,temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1,preds,1)
  return np.argmax(probas)

# Text generation loop

In [ ]:
import sys
import random

for epoch in range(1,60):
  print('epoch',epoch)
  # fit the model for 1 iteration
  model.fit(x,y, batch_size=128, epochs=1)
  start_index = random.randint(0,len(text)-maxlen-1)
  generated_text = text[start_index: start_index+maxlen]
  print('---Generated with seed: "' + generated_text + '"')


  # for different temperatures
  for temperature in [0.2,0.5,1.0,1.2]:
    print('temperature: ',temperature)
    sys.stdout.write(generated_text)
    for i in range(400):
      sampled = np.zeros((1,maxlen,len(chars)))
      for t,char in enumerate(generated_text):
        sampled[0,t,char_indices[char]] = 1.
      
      preds = model.predict(sampled,verbose=0)[0]
      next_index = sample(preds,temperature)
      next_char = chars[next_index]

      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)
